# 1. Processing Scraped Data

## Imports

In [ ]:
import pandas as pd
import re
import requests


# Path to input CSV file
input_csv_path = ''  # Replaced with csv outputs from WebScraper for each store

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(input_csv_path)

## Initial Cleaning of Raw Scraped Data

In [ ]:
import pandas as pd
import re

# Regex pattern to match price
price_pattern = re.compile(r'\$\d+\.\d+|Priced by add-ons')

# Initialize lists to hold item names and extracted prices
item_names = []
extracted_prices = []

current_item_name = None
current_item_price = None

for index, row in df.iterrows():
    # Check if the row could belong to a new item based on the 'Name.text' column
    if pd.notna(row['Name.text']) and row['Name.text'].strip():
        if current_item_name is not None:
            item_names.append(current_item_name)
            extracted_prices.append(current_item_price)

        current_item_name = row['Name.text']
        current_item_price = "No price info"

    if pd.notna(row['Price.text']) and str(row['Price.text']).strip():
        text_to_search = str(row['Price.text'])
        price_match = price_pattern.search(text_to_search)
        if price_match:
            current_item_price = price_match.group()

# Append the last item's details
if current_item_name is not None:
    item_names.append(current_item_name)
    extracted_prices.append(current_item_price)

# Create a new DataFrame with item names and their extracted prices
df_extract = pd.DataFrame({
    'Name': item_names,
    'Price': extracted_prices
})


# Filter out rows where 'Price' is "No price info"
cleaned_df = extract_df[extract_df['Price'] != "No price info"]

# Filter out rows where Price is 0 and "Name" column is not empty
df_filtered = cleaned_df[(cleaned_df['Price'] != 0) & (df['Name'].notna())]

# Drop duplicate rows based on the "Name" column
cleaned_df = df_filtered.drop_duplicates(subset=['Name'], keep='first')

# Save to CSV
output_csv_path = '/content/output.csv'
output_df.to_csv(output_csv_path, index=False)

output_csv_path



## Remove Non-Food Items

In [ ]:
# Path to input CSV
input_csv_path = ''  # Per store

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(input_csv_path)

# List of keywords to filter out
keywords = [
    'nail polish',
    'probiotics',
    "nature's way",
    'mineral fusion',
    'tints of nature',
    'Tissue',
    'Floral',
    'Water',
    'pads',
    'collagen',
    'kitchen',
    'bath',
    'garbage',
    'dishwasher',
    'diapers',
    'detergent',
    'air freshener',
    'wipes',
    'barbie',
    'doll',
    'craft',
    'odor',
    'laundry',
    'sponges',
    'gloves',
    'scented oil',
    'lint roller',
    'lotion',
    'tissue',
    'knife',
    'knives',
    'puppy',
    'dog',
    'cat',
    'clean',
    'pet',
    'refill',
    'dryer',
    'heavy duty',
    'soap',
    'sanitizer',
    'dish soap',
    'vacuum',
    'insect trap',
    'play set',
    'playset',
    'cotton',
    'dishwasher',
    'cleaning',
    'cloth',
    'sleep',
    'repellent',
    'robot',
    'laptop',
    'cleaner',
    'contraceptive',
    'hardwood',
    'scent',
    'pan',
    'multi surface',
    'surface',
    'pad',
    'dusting',
    'kit',
    'duster',
    'iRobot',
    'hefty',
    'plastic',
    'dish',
    'spoon',
    'dryer',
    'containers',
    'cat litter',
    'gadget',
    'toilet',
    'infusion',
    'fabric',
    'covid test',
    'cat litter',
    'bath tissue',
    'aluminum',
    'toilet',
    'laundry',
    'medicine',
    'cold remedy',
    'congestion',
    'water',
    'cold_remedy',
    'flu',
    'paper',
    'cough',
    'mucus',
    'Robitussin',
    'mucinex',
    'cold & flu'
]

# Normalize keywords for case-insensitive matching
normalized_keywords = [keyword.lower() for keyword in keywords]

# Function to determine if a row should be excluded based on keywords
def is_excluded(item_name):
    if isinstance(item_name, str):
        item_name_lower = item_name.lower()
        for keyword in normalized_keywords:
            if keyword in item_name_lower:
                return True
    return False

# Filter the DataFrame
cleaned_df = df[~df['Name'].apply(is_excluded)]

# Save the cleaned DataFrame to a new CSV file
cleaned_csv_path = '' # Updated per store
cleaned_df.to_csv(cleaned_csv_path, index=False)

print(f"Cleaned data saved to {cleaned_csv_path}")

## Query Census API & Calculate Initial $NRF_{9.3}$ Values For All Items

In [ ]:
def calculate_percent_dv(nutrient_value, nutrient_id):
    # RDIs for calculating %DV, expressed in the units provided by the API
    rdi_values = {
        '1003': 50,       # Protein in g
        '1079': 25,       # Fiber in g
        '1106': 900,      # Vitamin A in μg
        '1162': 90,       # Vitamin C in mg
        '1109': 15,       # Vitamin E in mg
        '1087': 1300,     # Calcium in mg
        '1089': 18,       # Iron in mg
        '1090': 400,      # Magnesium in mg
        '1092': 4700,     # Potassium in mg
        '1258': 20,       # Saturated Fat in g (limit)
        '2000': 50,       # Added Sugars in g (limit)
        '1093': 2300,     # Sodium in mg (limit)
    }
    return (nutrient_value / rdi_values.get(nutrient_id, 1)) * 100

def calculate_nrf93(food_nutrients):
    positive_nutrients = ['1003', '1079', '1106', '1162', '1109', '1087', '1089', '1090', '1092']
    negative_nutrients = ['1258', '2000', '1093']
    positive_sum = sum(calculate_percent_dv(n['value'], str(n['nutrientId'])) for n in food_nutrients if str(n['nutrientId']) in positive_nutrients)
    negative_sum = sum(calculate_percent_dv(n['value'], str(n['nutrientId'])) for n in food_nutrients if str(n['nutrientId']) in negative_nutrients)
    return positive_sum - negative_sum

def get_nrf93_score(food_name):
    API_KEY = '6ctPProwthJHdBqi73oO5ZO6CWLTHXGW616X90f9'
    BASE_URL = 'https://api.nal.usda.gov/fdc/v1/foods/search'
    params = {
        'query': food_name,
        'api_key': API_KEY,
        'dataType': ['Foundation', 'SR Legacy'],
        'pageSize': 2
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    scores = [calculate_nrf93(food['foodNutrients']) for food in data.get('foods', [])]
    return sum(scores) / len(scores) if scores else 0

# # Load the CSV file
df = pd.read_csv('/content/fast_food_pre_nrf.csv') # Completed by store type at first

# # Calculate the NRF9.3 score for each product name and store in a new column
df['NRF'] = df['Name'].apply(get_nrf93_score)

# # Save the updated DataFrame to a new CSV file
df.to_csv('fast_food_post_nrf_REAL.csv', index=False)

# 2. Associating Census Tracts to Stores With Google Maps API

## Imports

In [ ]:
!pip install simpledbf

In [ ]:
import os
import json
import string
import requests
import geopandas as gpd
import pandas as pd
from simpledbf import Dbf5
from shapely.geometry import Point, shape
from collections import defaultdict

## Import Census Tract Geographical Data Files & Visualize

In [ ]:
os.environ['SHAPE_RESTORE_SHX'] = 'YES'

# Read Shapefile
gdf = gpd.read_file('/content/sample_data/tl_2023_34_tract.shp')

# Display the DataFrame structure with non-geometry columns
dbf = Dbf5('/content/sample_data/tl_2023_34_tract.dbf')
df = dbf.to_dataframe()

# Merge DataFrame with your GeoDataFrame based on the index
gdf = gdf.merge(df, left_index=True, right_index=True)

gdf.plot()

# Mercer County Code = 021
mercer_county_fips_code = '021'

# Filter the GeoDataFrame to only include tracts from the specified county
filtered_gdf = gdf[gdf['COUNTYFP_x'] == mercer_county_fips_code]

# Test to confirm shape and info
filtered_gdf.info()
print(filtered_gdf.head())
filtered_gdf.plot()

## Query Google Maps API for Stores, Create JSON Before Adding FRCS Values

Query each tract around 5km radius except the 5 outliers to create stores_jason

In [ ]:
# Initialize Google Maps API key and store types
api_key = ''
store_types = ['grocery_or_supermarket', 'convenience_store', 'meal_takeaway']

# Hardcoded radii for specific tracts over 5000m in radius
specific_radii = {
    '34021003905': 5665,
    '34021003001': 5039,
    '34021003800': 8497,
    '34021003304': 5153,
    '34021004204': 6050,
}

all_stores_data = []

# API Request
def make_request(location, store_type, radius, api_key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': location,
        'radius': str(radius),  # Dynamic radius
        'type': store_type,
        'key': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json().get('results', [])
    else:
        print(f"API request failed for store type {store_type} with response: {response.text}")
        return []

# Query stores for each tract, around its geometry, based on store types
def find_stores_by_types(tract_id, tract_geometry, store_types, api_key):
    stores_found = []
    lat, lon = tract_geometry.centroid.y, tract_geometry.centroid.x
    location = f"{lat},{lon}"

    # Determine the radius to use
    radius = specific_radii.get(tract_id, 5000)  # Use specific radius if available, else default to 5000

    for store_type in store_types:
        results = make_request(location, store_type, radius, api_key)
        for result in results:
            store_name = result['name']
            stores_found.append({
                'name': store_name,
                'type': store_type,
            })
    return stores_found


query_gdf = gdf[gdf['COUNTYFP_x'] == mercer_county_fips_code]

# Loop through all
for index, row in query_gdf.iterrows():
    tract_id = row['GEOID_x']
    tract_stores = find_stores_by_types(tract_id, row['geometry'], store_types, api_key)

    # Structure to store current tract's store information in json
    tract_data = {
        "CensusTractID": tract_id,
        "StoreCount": len(tract_stores),
        "Stores": tract_stores
    }
    all_stores_data.append(tract_data)

# Convert to JSON
stores_json = json.dumps(all_stores_data, indent=4)
print(stores_json)

## Util Code for Finalizing JSON With Stores per Tract

Visualize data from stores_json to see what stores are missing or there by mistake

In [ ]:
# Load the JSON file
with open('/content/completed_mercer_county_tract_retailers.json') as f:
    data = json.load(f)

# Lists of observed stores
observed_stores = {
    "grocery_or_supermarket": ['Target', 'Wegmans', 'Shoprite', "McCaffrey's Food Market", 'Walmart',
                               'ALDI', 'Stop & Shop', 'ACME', 'Costco', "BJ's Wholesale Club"],
    "convenience_store": ['7-Eleven', 'Wawa', 'CVS', 'Walgreens'],
    "meal_takeaway": ["McDonald's", "Wendy's", 'Burger King', 'Taco Bell', "Domino's"]
}

# Initialize dictionaries to count occurrences of each store
store_counts_observed = {
    "grocery_or_supermarket": defaultdict(int),
    "convenience_store": defaultdict(int),
    "meal_takeaway": defaultdict(int)
}

store_counts_all = {
    "grocery_or_supermarket": defaultdict(int),
    "convenience_store": defaultdict(int),
    "meal_takeaway": defaultdict(int)
}

# Function to normalize store names and check for observed store names
def normalize_store_name(store_name, category):
    for observed_store in observed_stores[category]:
        if observed_store.lower() in store_name.lower():
            return observed_store
    return None

# Function to print the store counts as a table
def print_store_counts(category, store_counts):
    print(f"\n{category.replace('_', ' ').title()}")
    for store, count in store_counts[category].items():
        print(f"{store}: {count}")


# Loop through each tract and each store, normalize the store names, and count occurrences
for tract in data:
    for store in tract["Stores"]:
        store_type = store["type"]
        normalized_name = normalize_store_name(store["name"], store_type)
        if normalized_name:
            store_counts_observed[store_type][normalized_name] += 1
        else:
            store_counts_observed[store_type]["Unknown"] += 1
        # Count all stores including unobserved as their own entries
        store_counts_all[store_type][store["name"]] += 1

# Print total and observed counts
for category in store_counts_observed.keys():
    total_stores = sum(store_counts_all[category].values())
    observed_stores = sum(store_counts_observed[category].values()) - store_counts_observed[category]["Unknown"]
    print(f"\nCategory: {category.replace('_', ' ').title()}")
    print(f"Total Stores: {total_stores}")
    print(f"Observed Stores: {observed_stores}")

# Print tables where unobserved stores are aggregated under "Unknown"
print("\nFirst set of tables (Unobserved stores counted as 'Unknown'):")
for category in store_counts_observed.keys():
    print_store_counts(category, store_counts_observed)

# Print tables listing each unique store name, including unobserved stores
print("\nSecond set of tables (All stores listed individually):")
for category in store_counts_all.keys():
    print_store_counts(category, store_counts_all)


Temporarily add centroids for each tract to manually search missing stores with observed data

In [ ]:
# Load your JSON data
with open("/content/mercer_county_tract_retailers.json", "r") as f:
    tract_data = json.load(f)

# Modified loop to add centroid at the beginning of each tract entry
for i, tract in enumerate(tract_data):
    # Find the corresponding row in query_gdf
    tract_gdf_row = query_gdf[query_gdf['GEOID_x'] == tract["CensusTractID"]].iloc[0]

    # Calculate the centroid
    centroid = tract_gdf_row.geometry.centroid
    lat, lon = centroid.y, centroid.x

    # Reconstruct the tract entry with centroid at the beginning
    new_tract_data = {
        "CensusTractID": tract["CensusTractID"],
        "centroid": {"latitude": lat, "longitude": lon},
        "StoreCount": tract["StoreCount"],
        "Stores": tract["Stores"]
    }

    # Update the original list with the new tract data
    tract_data[i] = new_tract_data

# Print the modified JSON data
print(json.dumps(tract_data, indent=4))


Manually search for missing stores by name and add to JSON and export

In [ ]:
# Load existing data
with open("/content/temp_centroids_added.json", "r") as f:
    tracts_data = json.load(f)

# Define the stores to query
store_queries = {
    "grocery_or_supermarket": ["Stop & Shop", "Costco Wholesale", "BJs Wholesale Club", "McCaffrey's Food Market-Princeton", "McCaffrey's Food Market - West Windsor", "Walmart", "Walmart Supercenter"],
    "meal_takeaway": ["McDonald", "Wendy", "Burger King"]
}

# Removes punctuation and convert to lowercase
def normalize_name(name):
    remove_punctuation = str.maketrans('', '', string.punctuation)
    return name.lower().translate(remove_punctuation).strip()

# Checks for close match
def is_close_match(result_name, query_names):
    normalized_result_name = normalize_name(result_name)
    for query_name in query_names:
        normalized_query_name = normalize_name(query_name)
        if normalized_query_name in normalized_result_name and \
           len(normalized_result_name) <= len(normalized_query_name) + 2:
            return True
    return False


# Check if the store is already listed and adds if not
def add_missing_stores(tract, api_key, store_queries):
    added_stores = []
    for category, store_names in store_queries.items():
        for store_name in store_names:
            location = f"{tract['centroid']['latitude']},{tract['centroid']['longitude']}"
            radius = "5000"
            url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
            params = {
                "location": location,
                "radius": radius,
                "keyword": store_name,
                "key": api_key
            }
            response = requests.get(url, params=params)
            if response.status_code == 200:
                results = response.json().get("results", [])
                for result in results:
                    if is_close_match(result["name"], [store_name]):
                        store_data = {
                            "name": result["name"],
                            "type": category
                        }
                        if store_data not in tract["Stores"]:
                            tract["Stores"].append(store_data)
                            added_stores.append(store_data)
                            print(f"Added {store_data['name']} to Tract ID {tract['CensusTractID']}")
    return added_stores


# Iterate over each tract and add missing stores
for tract in tracts_data:
    add_missing_stores(tract, api_key, store_queries)

# Save the updated data back to JSON
with open("completed_mercer_county_tract_retailers.json", "w") as f:
    json.dump(tracts_data, f, indent=4)

print("Updated JSON saved.")


# 3. Clustering and Predicting Labels

## Imports and initial checks

Initial checks include printing figure that shows Average FRCS by category vs $\Theta$

In [ ]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from sklearn.metrics import silhouette_score
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline


# Hardcoded data
avg_nrf_all = 15.94365475
max_nrf_all = 74.41707905
min_nrf_all = -24.7830702
avg_cost_all = 5.602255578
max_cost_all = 12.1995
min_cost_all = 1.65

# Store hardcoded data values for stores
stores_data_dict = {
    'Store': ['McCaffrey', 'ACME', 'ALDI', 'BJ', 'Costco', 'ShopRite', 'Stop & Shop', 'Target', 'Walmart', 'Wegman',
              'Wawa', 'CVS', '7-Eleven', 'Walgreens',
              "Domino's", 'Taco Bell', 'McDonald', 'Wendy', 'Burger King'],
    'Category': ['grocery_or_supermarket']*10 + ['convenience_store']*4 + ['meal_takeaway']*5,
    'Avg NRF': [20.2801, 16.8792, 19.2794, 18.8869, 25.1476, 12.1907, 20.3024, 13.7764, 15.3831, 22.7182,
                9.8079, -7.2559, -8.8167, 2.0326,
                -7.4033, -5.2312, -11.1748, -6.9643, -11.8451],
    'Avg Cost': [6.7002, 5.4859, 3.7146, 6.2001, 6.6022, 5.2021, 6.4960, 5.4342, 4.1983, 6.3191,
                 4.7611, 6.3314, 4.3121, 5.3588,
                 10.9794, 7.2684, 6.4656, 6.1878, 5.5602],
}

# Convert to Dataframe
stores_data = pd.DataFrame(stores_data_dict)


# Load JSON data with tract data
with open("/content/sample_data/Mercer_country_tract_retailers_pre_frcs.json", "r") as file:
    data = json.load(file)



# Function to calculate FRCS
def calculate_frcs(row, theta):
    w1 = theta
    w2 = 1 - theta
    nrf_component = (row['Avg NRF'] - avg_nrf_all) / (max_nrf_all - min_nrf_all)
    cost_component = (row['Avg Cost'] - avg_cost_all) / (max_cost_all - min_cost_all)
    return w1 * nrf_component - w2 * cost_component

# Function to update json with frcs values given theta
def update_json_with_frcs(theta, data):
    print('starting update_json_with_frcs for theta ', theta)

    # Recalculate FRCS scores based on new theta
    for index, row in stores_data.iterrows():
        stores_data.at[index, 'FRCS'] = calculate_frcs(row, theta)

    # Calculate mean and sd FRCS for each Category
    category_stats = stores_data.groupby('Category')['FRCS'].agg(['mean', 'std'])
    print('category stats ',category_stats)

    # Update the JSON data with these FRCS values
    index = 0
    for tract in data:
        frcs_values = []
        print(f'Processing tract {index + 1} of {len(data)}')
        index = index + 1
        for store in tract['Stores']:
            store_name = store["name"].lower()
            store_assigned = False
            for _, store_row in stores_data.iterrows():
                if store_row["Store"].lower() in store_name:
                    store["FRCS"] = store_row["FRCS"]
                    store_assigned = True
                    break

            if not store_assigned:
                category = store["type"]
                if category in category_stats.index:
                    mean = category_stats.loc[category, 'mean']
                    std = category_stats.loc[category, 'std']
                    store["FRCS"] = np.random.normal(mean, std)
                else:
                    print(f"No category stats for {store['name']} of type {store['type']}, assigning default FRCS")

            frcs_values.append(store["FRCS"])

        if frcs_values:
            average_frcs = np.average(frcs_values)
            tract['average FRCS'] = average_frcs

    print('done')

# Function to find optimal k by plotting Silhouette scores and WCSS
def find_optimal_k_and_plot_3D(data, theta_values):
    # Initialize DataFrames to store WCSS and silhouette scores
    wcss_df = pd.DataFrame(index=range(2, 11), columns=theta_values)
    silhouette_df = pd.DataFrame(index=range(2, 11), columns=theta_values)

    for theta in theta_values:
        update_json_with_frcs(theta, data)

        # Extract 'StoreCount' and 'average FRCS' for each tract
        X_data = np.array([(tract['StoreCount'], tract['average FRCS']) for tract in data])

        for k in range(2, 11):
            print('k ', k, ' theta ', theta)
            kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
            kmeans.fit(X_data)
            # Store WCSS and silhouette score for each k and theta
            wcss_df.at[k, theta] = kmeans.inertia_
            silhouette_df.at[k, theta] = silhouette_score(X_data, kmeans.labels_)


        # Plotting the elbow method and silhouette scores with highlights for chosen k
        plt.figure(figsize=(14, 7))


    # Plotting 3D plots for WCSS and silhouette scores
    fig = plt.figure(figsize=(16, 8))

    # WCSS 3D Plot
    ax1 = fig.add_subplot(121, projection='3d')
    X, Y = np.meshgrid(theta_values, range(2, 11))
    Z = wcss_df.values
    ax1.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')
    ax1.set_title('WCSS across Theta and k')
    ax1.set_xlabel('Theta')
    ax1.set_ylabel('k')
    ax1.set_zlabel('WCSS')

    # Silhouette Scores 3D Plot
    ax2 = fig.add_subplot(122, projection='3d')
    Z = silhouette_df.values
    ax2.plot_surface(X, Y, Z, cmap='plasma', edgecolor='none')
    ax2.set_title('Silhouette Scores across Theta and k')
    ax2.set_xlabel('Theta')
    ax2.set_ylabel('k')
    ax2.set_zlabel('Silhouette Score')

    plt.show()

    X, Y = np.meshgrid(wcss_df.columns, wcss_df.index)  # Theta values and k values
    Z_wcss = wcss_df.values
    Z_silhouette = silhouette_df.values

    # Plot for WCSS
    fig_wcss = go.Figure(data=[go.Surface(z=Z_wcss, x=X, y=Y)])
    fig_wcss.update_layout(title='WCSS across Theta and k', autosize=False,
                          scene=dict(
                              xaxis_title='Theta',
                              yaxis_title='k',
                              zaxis_title='WCSS'),
                          width=700, height=700,
                          margin=dict(l=65, r=50, b=65, t=90))
    fig_wcss.show()

    # Plot for Silhouette Scores
    fig_silhouette = go.Figure(data=[go.Surface(z=Z_silhouette, x=X, y=Y, colorscale='Viridis')])
    fig_silhouette.update_layout(title='Silhouette Scores across Theta and k', autosize=False,
                                scene=dict(
                                    xaxis_title='Theta',
                                    yaxis_title='k',
                                    zaxis_title='Silhouette Score'),
                                width=700, height=700,
                                margin=dict(l=65, r=50, b=65, t=90))
    fig_silhouette.show()





def add_census_data_to_tract(tract):
    # Base URL for the Census API
    base_url = "https://api.census.gov/data"

    # Set year and dataset
    year = "2022"
    dataset = "acs/acs5"

    # Variables for API request
    variables = "B19013_001E,B17001_002E,B22010_001E,B19083_001E,B23025_005E,B15003_002E,B16010_041E,B15003_022E,B25077_001E,B08301_010E,B25044_003E,B01003_001E,B02001_003E,B03002_012E,B02001_005E,B02001_004E,B02001_006E,B01001_001E"


    variable_labels = [
        "MedianIncome", "PovertyRate", "HHWithSNAP", "Inequality", "Unemployment",
        "BelowHighSchool", "CollegeNoDegree", "BachelorsOrMore", "PropertyValue",
        "PublicTransport", "NoVehicle", "Population", "Black", "Hispanic",
        "Asian", "NativeAmerican", "PacificIslander",
    ]

    median_values = {
        "MedianIncome": 88921.50,
        "PovertyRate": 0.07,
        "HHWithSNAP": 1657.50,
        "Inequality": 0.41,
        "Unemployment": 0.03,
        "BelowHighSchool": 0.00,
        "CollegeNoDegree": 0.26,
        "BachelorsOrMore": 0.18,
        "PropertyValue": 292050.00,
        "PublicTransport": 0.02,
        "NoVehicle": 0.01,
        "Population": 4581.50,
        "Black": 0.12,
        "Hispanic": 0.10,
        "Asian": 0.05,
        "NativeAmerican": 0.00,
        "PacificIslander": 0.00
    }

    # Assuming tract_code is correctly set
    print(tract["CensusTractID"])
    state_code = tract["CensusTractID"][:2]
    county_code = tract["CensusTractID"][2:5]
    tract_code = tract["CensusTractID"][5:]


    url = f"{base_url}/{year}/{dataset}?get={variables}&for=tract:{tract_code}&in=state:{state_code}+county:{county_code}"
    print(url)
    response = requests.get(url)


    if response.status_code == 200:
        data = response.json()[1]  # First element is header, second is data
        total_population = float(data[variable_labels.index("Population")])

        for i, label in enumerate(variable_labels):
            try:
                value = float(data[i])
                # Check if the label is among those that need to be converted to percentages
                if label in ["Unemployment", "PovertyRate", "BelowHighSchool", "CollegeNoDegree", "BachelorsOrMore", "PublicTransport", "NoVehicle", "Black", "Hispanic", "Asian", "NativeAmerican", "PacificIslander"]:
                    # Calculate percentage and constrain to non-negative values
                    percent_value = max(0, (value / total_population))
                    # Replace with median if original value is less than or equal to zero
                    tract[label] = round(percent_value, 4) if value > 0 else median_values[label]
                    if value < 0:
                        print('negative ', label, ' averted')
                else:
                    # For non-percentage values, replace with median if value is less than or equal to zero
                    tract[label] = value if value > 0 else median_values[label]
            except ValueError:
                tract[label] = median_values[label]
            except ZeroDivisionError:
                tract[label] = median_values[label]
    # Print each label and its value for verification
    for label in variable_labels:
        print(label, tract[label])

    # Return the updated tract dictionary
    return tract

# Function to add census data for all tracts
def add_all_census_data(data, stores_data):
    theta_values=np.linspace(0, 1, 21)
    tracts_data = []
    for index, tract in enumerate(data):

        print(f'Processing tract {index + 1} of {len(data)}')
        add_census_data_to_tract(tract)

    return "done"


# Prepares features for K-Means clustering
def prepare_data_for_clustering(data):
    # Initialize lists to hold the features
    features = []

    for tract in data:
        store_count = tract['StoreCount']
        average_frcs = tract['average FRCS']

        tract_features = [store_count, average_frcs]
        features.append(tract_features)

    # Convert features to NumPy arrays
    features_array = np.array(features)

    return features_array


# Preparing features for Random Forest classification
def prepare_data_for_rf(data):
    features = []

    for tract in data:
        tract_features = [
            float(tract.get('MedianIncome')),
            float(tract.get('PovertyRate')),
            float(tract.get('HHWithSNAP')),
            float(tract.get('Inequality')),
            float(tract.get('Unemployment')),
            float(tract.get('BelowHighSchool')),
            float(tract.get('CollegeNoDegree')),
            float(tract.get('BachelorsOrMore')),
            float(tract.get('PropertyValue')),
            float(tract.get('PublicTransport')),
            float(tract.get('NoVehicle')),
            float(tract.get('Population')),
            float(tract.get('Black')),
            float(tract.get('Hispanic')),
            float(tract.get('Asian')),
            float(tract.get('NativeAmerican')),
            float(tract.get('PacificIslander'))
        ]
        features.append(tract_features)


    # Convert features list to NumPy array
    features_array = np.array(features)

    return features_array

def plot_k_theta_accuracy(data):
    theta_values = np.linspace(0, 1, 21)
    k_values = range(2, 12)
    accuracies = np.zeros((len(theta_values), len(k_values)))

    for i, theta in enumerate(theta_values):
        print('Processing for theta:', theta)
        update_json_with_frcs(theta, data)
        features = prepare_data_for_clustering(data)

        for j, k in enumerate(k_values):
              kmeans = KMeans(n_clusters=k, random_state=42).fit(features)
              labels = kmeans.labels_

              features_rf = prepare_data_for_rf(data)

              # Check if any class has fewer members than n_splits
              if np.any(np.bincount(labels) < 5):  # Using 5 as the n_splits for StratifiedKFold
                  print("Falling back to simple train-test split due to small class size.")
                  X_train, X_test, y_train, y_test = train_test_split(features_rf, labels, test_size=0.3, random_state=42)
                  rf = RandomForestClassifier(n_estimators=100, random_state=42)
                  rf.fit(X_train, y_train)
                  y_pred = rf.predict(X_test)
                  accuracy = accuracy_score(y_test, y_pred)
              else:
                  # Continue With StratifiedKFold and SMOTE if applicable
                  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                  accuracies_skf = []

                  for train_index, test_index in skf.split(features_rf, labels):
                      X_train, X_test = features_rf[train_index], features_rf[test_index]
                      y_train, y_test = labels[train_index], labels[test_index]

                      # Dynamically set SMOTE's k_neighbors parameter
                      #smote_k_neighbors = min(len(np.unique(y_train)) - 1, np.bincount(y_train).min() - 1, 5)
                      smote_k_neighbors = min([y_train.tolist().count(class_label) - 1 for class_label in np.unique(y_train)])
                      smote_k_neighbors = max(1, smote_k_neighbors)  # Ensure k_neighbors is at least 1

                      smote = SMOTE(random_state=42, k_neighbors=smote_k_neighbors)

                      rf = RandomForestClassifier(n_estimators=100, random_state=42)
                      pipeline = make_pipeline(smote, rf)

                      pipeline.fit(X_train, y_train)
                      y_pred = pipeline.predict(X_test)

                      accuracies_skf.append(accuracy_score(y_test, y_pred))
                  accuracy = np.mean(accuracies_skf)
              accuracies[i, j] = accuracy
              print(f"Average Accuracy for theta {theta:.2f} and k {k}: {accuracy:.4f}")

    # Plotting code
    k_values_list = list(k_values)
    T, K = np.meshgrid(np.array(theta_values), np.array(k_values_list))
    Z = accuracies.T  # Transpose for Plotly

    indices = [int(i * (len(theta_values) - 1)) for i in [0, 0.25, 0.5, 0.75, 1]]
    ticktext = [f"{theta_values[i]:.2f}" for i in indices]


    fig = go.Figure(data=[go.Surface(z=Z, x=T, y=K, colorscale='Viridis')])
    fig.update_layout(
        title='Accuracy across Theta and k',
        scene=dict(
            xaxis=dict(
                title='Theta',
                tickvals=indices,
                ticktext=ticktext
            ),
            yaxis=dict(
                title='k',
                tickvals=list(range(len(k_values_list))),
                ticktext=[str(k) for k in k_values_list]
            ),
            zaxis=dict(title='Accuracy'),
        ),
        autosize=True
    )
    fig.show()
    return Z



# Average FRCS of grocery and convenience stores
def f_grocery(theta):
    return np.interp(theta, avg_frcs_grocery.index, avg_frcs_grocery.values)
def f_convenience(theta):
    return np.interp(theta, avg_frcs_convenience.index, avg_frcs_convenience.values)

# Find intersection
def find_intersection(theta):
    return f_grocery(theta) - f_convenience(theta)



df_stores = pd.DataFrame(stores_data)

# Calculate FRCS for each store at each theta
theta_values = np.linspace(0, 1, num=101)

frcs_values = pd.DataFrame(index=df_stores.index, columns=theta_values)

# Calculate the FRCS for each store and each theta value
for theta in theta_values:
    frcs_values[theta] = df_stores.apply(calculate_frcs, axis=1, args=(theta,))

# Add the store names and categories to the frcs_values DataFrame for reference
frcs_values['Store'] = df_stores['Store']
frcs_values['Category'] = df_stores['Category']

# Split the DataFrame into categories
grocery_stores = frcs_values[frcs_values['Category'] == 'grocery_or_supermarket']
convenience_stores = frcs_values[frcs_values['Category'] == 'convenience_store']
fast_food_stores = frcs_values[frcs_values['Category'] == 'meal_takeaway']

# Calculate the average FRCS for each category across all theta values
avg_frcs_grocery = grocery_stores[theta_values].mean()
avg_frcs_convenience = convenience_stores[theta_values].mean()
avg_frcs_fast_food = fast_food_stores[theta_values].mean()

# Use fsolve to find the intersection point, starting the search at theta=0.5
theta_intersection = fsolve(find_intersection, 0.5)[0]
frcs_intersection = f_grocery(theta_intersection)


# Print the intersection point
print(f"The lines intersect at theta = {theta_intersection:.2f} with an average FRCS of {frcs_intersection:.2f}")


# Plotting the intersection point
plt.figure(figsize=(10, 5))

# Plot each category
plt.plot(avg_frcs_grocery.index, avg_frcs_grocery.values, marker='o', linestyle='-', color='black', label='Grocery Store')
plt.plot(avg_frcs_convenience.index, avg_frcs_convenience.values, marker='x', linestyle='--', color='grey', label='Convenience Store')
plt.plot(avg_frcs_fast_food.index, avg_frcs_fast_food.values, marker='^', linestyle='-.', color='darkgrey', label='Fast Food')

# Mark the intersection point with a star
plt.plot(theta_intersection, frcs_intersection, marker='*', markersize=10, color='red', label='Intersection')

# Add titles and labels
plt.title('Average FRCS Across Observed Retailers by Categories as a Function of Theta')
plt.xlabel('Theta')
plt.ylabel('Average FRCS')
plt.legend()
plt.grid(True)
plt.show()


print('Imports Completed, Data & Functions Loaded')

## Create plots to analyze optimal K with WCSS and Silhouette Scores

In [ ]:
find_optimal_k_and_plot_3D(data, np.linspace(0, 1, 11))

## Add all Census Data to JSON

In [ ]:
print(add_all_census_data(data, stores_data))

## Create plot to analyze K and $\Theta$ for accuracy using RK/SMOTE/StratifiedKFold

Util code to fix axis values for plot

In [ ]:
# Call the plotting function
Z = plot_k_theta_accuracy(data)

# Fix plot
theta_values = np.linspace(0, 1, 21)
k_values = range(2, 12)
k_values_list = list(k_values)
T, K = np.meshgrid(range(len(theta_values)), range(len(k_values)))

# Indices for specific theta values (0, 0.25, 0.5, 0.75, 1)
indices = [int(i * (len(theta_values) - 1)) for i in [0, 0.25, 0.5, 0.75, 1]]
ticktext = [f"{theta_values[i]:.2f}" for i in indices]

fig = go.Figure(data=[go.Surface(z=Z, x=T, y=K, colorscale='Viridis')])
fig.update_layout(
    title='Accuracy across Theta and k',
    scene=dict(
        xaxis=dict(
            title='Theta',
            tickvals=indices,
            ticktext=ticktext
        ),
        yaxis=dict(
            title='k',
            tickvals=list(range(len(k_values))),
            ticktext=[str(k) for k in k_values]
        ),
        zaxis=dict(title='Accuracy'),
    ),
    autosize=True
)

fig.show()


## Check Value at $\Theta=0.66$, $K=3$, Plot Cluster Plot

In [ ]:
def evaluate_model_at_theta_k(data, theta):
    update_json_with_frcs(theta, data)
    features = prepare_data_for_clustering(data)

    # Apply K-Means clustering with k=3
    kmeans = KMeans(n_clusters=3, random_state=42).fit(features)
    labels = kmeans.labels_

    # Define cluster labels
    cluster_labels = {0: 'Food Swamp', 1: 'Food Oasis', 2: 'Food Desert'}  # Update as necessary

    # Plotting
    plt.figure(figsize=(10, 6))
    colors = ['green', 'blue', 'orange']
    for cluster in range(3):
        clustered_df = df[labels == cluster]
        # Use cluster_labels for legend
        plt.scatter(clustered_df['StoreCount'], clustered_df['AverageFRCS'], color=colors[cluster], label=cluster_labels[cluster], alpha=0.6, edgecolor='black')

    plt.title('K-Means Clustering of Census Tracts')
    plt.xlabel('Store Count')
    plt.ylabel('Average FRCS')
    plt.legend()
    plt.grid(True)
    plt.show()

    features_rf = prepare_data_for_rf(data)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    feature_importances_list = []

    all_true_labels = []
    all_predictions = []
    feature_names = [
        "MedianIncome", "PovertyRate", "HHWithSNAP", "Inequality", "Unemployment",
        "BelowHighSchool", "CollegeNoDegree", "BachelorsOrMore", "PropertyValue",
        "PublicTransport", "NoVehicle", "Population", "Black", "Hispanic",
        "Asian", "NativeAmerican", "PacificIslander"
    ]


    for train_index, test_index in skf.split(features_rf, labels):
        X_train, X_test = features_rf[train_index], features_rf[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # Dynamically adjust SMOTE's k_neighbors parameter
        smote_k_neighbors = min([y_train.tolist().count(class_label) - 1 for class_label in np.unique(y_train)])
        smote_k_neighbors = max(1, smote_k_neighbors)  # Ensure k_neighbors is at least 1


        smote = SMOTE(random_state=42, k_neighbors=smote_k_neighbors)
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        pipeline = Pipeline([('smote', smote), ('rf', rf)])

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        accuracies.append(accuracy_score(y_test, y_pred))
        all_true_labels.extend(y_test)
        all_predictions.extend(y_pred)

        rf_feature_importances = pipeline.named_steps['rf'].feature_importances_
        feature_importances_list.append(rf_feature_importances)

    average_accuracy = np.mean(accuracies)
    print("Average Accuracy:", average_accuracy)
    print("Accuracy per fold:", accuracies)

    average_feature_importances = np.mean(feature_importances_list, axis=0)
    feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': average_feature_importances}).sort_values(by='Importance', ascending=False)
    print("Average Feature Importances across folds:\n", feature_importances_df)

    print("Classification Report for All Folds:")
    class_names = ["Food Swamp", "Food Oasis", "Food Desert"]
    print(classification_report(all_true_labels, all_predictions, target_names=class_names))


evaluate_model_at_theta_k(data, 0.66)


## Print Feature Statistics and Summary Table

In [ ]:
def print_feature_statistics(features_array, feature_names):
    # Ensure features_array is a NumPy array
    features_array = np.array(features_array)

    # Iterate over each feature by index and name
    for i, feature_name in enumerate(feature_names):
        # Calculate mean and standard deviation for the current feature across all tracts
        feature_mean = np.mean(features_array[:, i])
        feature_std = np.std(features_array[:, i])
        feature_min = np.min(features_array[:, i])
        feature_max = np.max(features_array[:, i])
        feature_median = np.median(features_array[:, i])

        #  Min = {feature_min:.2f}, Max = {feature_max:.2f}, Median = {feature_median:.2f}"
        # Print the statistics
        print(f"{feature_name}: Mean = {feature_mean:.2f}, SD = {feature_std:.2f}, max = {feature_max:.2f}, min = {feature_min:.2f}")

# Example feature names
feature_names = [
    "MedianIncome", "PovertyRate", "HHWithSNAP", "Inequality",
    "Unemployment", "BelowHighSchool", "CollegeNoDegree",
    "BachelorsOrMore", "PropertyValue", "PublicTransport",
    "NoVehicle", "Population", "Black", "Hispanic",
    "Asian", "NativeAmerican", "PacificIslander"
]

print_feature_statistics(prepare_data_for_rf(data), feature_names)

features_array = prepare_data_for_rf(data)
features = prepare_data_for_clustering(data)

# Apply K-Means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42).fit(features)
labels = kmeans.labels_


# Example feature names
feature_names = [
    "MedianIncome", "PovertyRate", "HHWithSNAP", "Inequality",
    "Unemployment", "BelowHighSchool", "CollegeNoDegree",
    "BachelorsOrMore", "PropertyValue", "PublicTransport",
    "NoVehicle", "Population", "Black", "Hispanic",
    "Asian", "NativeAmerican", "PacificIslander"
]

def print_feature_statistics_by_classification(features_array, feature_names, labels):
    df = pd.DataFrame(features_array, columns=feature_names)
    df['Classification'] = labels

    summary_table = pd.DataFrame()

    for feature_name in feature_names:
        summary = df.groupby('Classification')[feature_name].agg(['mean', 'std']).apply(lambda x: f"{x['mean']:.2f} ({x['std']:.2f})", axis=1)
        summary_table[feature_name] = summary

    return summary_table

summary_table = print_feature_statistics_by_classification(features_array, feature_names, labels)
print(summary_table)

## Plot Map of Mercer County with Labels for $k=3$,$\Theta=0.66$

In [ ]:
import matplotlib.patches as mpatches

# Prepare DataFrame for clustering
tract_info = []
for item in data:
    tract_info.append({
        "CensusTractID": item["CensusTractID"],
        "StoreCount": item["StoreCount"],
        "AverageFRCS": item["average FRCS"]  # Using get() to avoid KeyError if missing
    })

df_tracts = pd.DataFrame(tract_info)

# Clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_tracts['Cluster'] = kmeans.fit_predict(df_tracts[['StoreCount', 'AverageFRCS']])

# Map CensusTractID to GEOID_x for merging
df_tracts['GEOID_x'] = df_tracts['CensusTractID']

# Merge the cluster information back to your GeoDataFrame
merged_gdf = filtered_gdf.merge(df_tracts[['GEOID_x', 'Cluster']], on='GEOID_x')

# Prep Plot
fig, ax = plt.subplots(figsize=(10, 6))

# Define custom labels and colors for clusters
cluster_labels = {0: 'Food Swamp', 1: 'Food Oasis', 2: 'Food Desert'}
cluster_colors = ['green', 'blue', 'orange']

# Plot each cluster with its color and label
for cluster, color in zip(cluster_labels.keys(), cluster_colors):
    merged_gdf.loc[merged_gdf['Cluster'] == cluster].plot(color=color, ax=ax, label=cluster_labels[cluster])

# Create custom legend
legend_patches = [mpatches.Patch(color=color, label=label) for color, label in zip(cluster_colors, cluster_labels.values())]
ax.legend(handles=legend_patches, title="Classification")

plt.axis('off')
plt.show()
